In [2]:
"""

1. Download MERRA2 data at daily interval

2. Save as NetCDF4
    - Snow depth 

"""

# Import libraries
import glob
import os
from datetime import datetime
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4
import matplotlib.pyplot as plt

# Define user
user = 'jryan4'

# Define base path
path = '/Users/' + user + '/Dropbox (University of Oregon)/research/snowfall/'

# Define path to links
links = pd.read_csv(path + 'data/links/subset_M2T3NXGLC_5.12.4_20230327_223103_.txt', 
                    skiprows=0, sep='\t', header=None)
links.rename(columns={0: "link"}, inplace=True)
links['year'] = links['link'].str[119:123]
links['month'] = links['link'].str[162:164].astype(int)

# Define years
years = np.arange(1980, 2001)

# Filter March
links_summer = links[(links['month'] > 2) & (links['month'] < 10)]

# Read MERRA-2 sample
merra = xr.open_dataset(path + 'data/merra_sample/MERRA2_200.tavg1_2d_int_Nx.20000101.SUB.nc')

In [3]:
%%capture
# Loop over every link and append a 1D array of ablation zone temperature + time
for year in years:
    
    if os.path.exists(path + 'data/merra_snowdepth/sd_' + str(year) + '.nc'):
        print(f'Skipping...{str(year)}')
    else:
        print(f'Processing...{str(year)}')

        # Make a new DataFrame
        link_year = links_summer[links_summer['year'] == str(year)]

        t = []
        t2m = []
        for j in range(len(link_year)):

            # Index link
            link = '"' + str(link_year.iloc[j].values[0]) + '"'

            # Download MERRA2 using WGET
            !wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --keep-session-cookies --no-check-certificate --content-disposition $link --directory-prefix=tmp_sd -nd

            # Import temporary file
            merra = xr.open_dataset(sorted(glob.glob(path + 'repo/1-download/tmp_sd/*.nc'))[0])
            
            # Clean up temporary files
            files = glob.glob(path + 'repo/1-download/tmp_sd/*.nc')
            for f in files:
                os.remove(f)

            # Calculate daily mean
            t2m_resample = np.mean(merra['SNOWDP_GL'], axis=0).values

            # Append to list
            t2m.append(t2m_resample)
            t.append(merra['time'].values[0].astype('datetime64[D]'))

        # Save as NetCDF
        ds_data = xr.Dataset(
        data_vars={
            "sd": (("time", "lat", "lon"), np.array(t2m).astype('float32')),
        },

        coords={
            "time": pd.DatetimeIndex(np.array(t), freq='D'),
            "longitude": (('lon',), merra['lon'].values),
            "latitude": (('lat',), merra['lat'].values),    
        },

        attrs={
            "Produced": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Units": 'K',
            "Author":'Johnny Ryan', 
            "Email":'jryan4@uoregon.edu'
        },
        )

        # Save
        ds_data.to_netcdf(path + 'data/merra_snowdepth/sd_' + str(year) + '.nc')
        